In [1]:
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential

# 数据集目录
dataset_dir = 'C:/tester01'

# 输出目录
output_dir = 'C:/caix tester01'

# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

# 子文件夹名称
categories = ['cataract', 'normal', 'surgery']

# 每个类别的划分比例
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# 创建训练、验证和测试子目录
for category in categories:
    for subset in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_dir, category, subset), exist_ok=True)

# 划分数据集
for category in categories:
    category_dir = os.path.join(dataset_dir, category)
    output_category_dir = os.path.join(output_dir, category)

    # 列出每个类别下的所有图像文件
    image_files = os.listdir(category_dir)

    # 随机打乱图像文件列表
    random.shuffle(image_files)

    # 计算划分的索引
    total_images = len(image_files)
    train_split = int(total_images * train_ratio)
    val_split = int(total_images * (train_ratio + val_ratio))

    # 划分数据集
    train_files = image_files[:train_split]
    val_files = image_files[train_split:val_split]
    test_files = image_files[val_split:]

    # 移动图像文件到对应的目录
    for filename in train_files:
        src = os.path.join(category_dir, filename)
        dst = os.path.join(output_category_dir, 'train', filename)
        shutil.copy(src, dst)

    for filename in val_files:
        src = os.path.join(category_dir, filename)
        dst = os.path.join(output_category_dir, 'val', filename)
        shutil.copy(src, dst)

    for filename in test_files:
        src = os.path.join(category_dir, filename)
        dst = os.path.join(output_category_dir, 'test', filename)
        shutil.copy(src, dst)

# 图像大小
image_size = (150, 150)
batch_size = 32

# 创建数据生成器
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
)

train_generator = train_datagen.flow_from_directory(
    output_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    output_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# 构建卷积神经网络模型
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(3, activation='softmax')  
])

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# 评估模型性能
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    output_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy}")

Found 2453 images belonging to 3 classes.
Found 611 images belonging to 3 classes.
Epoch 1/10
77/77 [==============================] - 25s 311ms/step - loss: 0.6442 - accuracy: 0.7574 - val_loss: 0.3169 - val_accuracy: 0.8838
Epoch 2/10
77/77 [==============================] - 25s 323ms/step - loss: 0.2413 - accuracy: 0.9230 - val_loss: 0.2082 - val_accuracy: 0.9378
Epoch 3/10
77/77 [==============================] - 26s 342ms/step - loss: 0.1610 - accuracy: 0.9486 - val_loss: 0.1621 - val_accuracy: 0.9542
Epoch 4/10
77/77 [==============================] - 24s 305ms/step - loss: 0.1279 - accuracy: 0.9568 - val_loss: 0.1773 - val_accuracy: 0.9345
Epoch 5/10
77/77 [==============================] - 24s 313ms/step - loss: 0.1009 - accuracy: 0.9641 - val_loss: 0.1588 - val_accuracy: 0.9394
Epoch 6/10
77/77 [==============================] - 24s 305ms/step - loss: 0.0717 - accuracy: 0.9702 - val_loss: 0.1316 - val_accuracy: 0.9542
Epoch 7/10
77/77 [==============================] - 24s 313

In [2]:
model.save('caix.h5')